In [7]:
from cassandra.cluster import Cluster
cluster = Cluster(["172.17.0.2"])
session = cluster.connect("tfbda")
query = 'select ano, count(mes), qtd_obt,sum(vlr_bnf), cod_mun FROM tfbda.bsafm_obt GROUP BY ano, cod_mun'
# qtd_obt jah eh a mesma por conta do dado ser anual
# rows = session.execute('SELECT ano, mes, count(*) FROM bsa_fam GROUP BY ano, mes'210055)
# rows = session.execute('SELECT * FROM bsa_fam obt_inf where bsa_fam.cod_mun = obt_inf.cod_mun ALLOW FILTERING')
# rows = session.execute('SELECT * FROM obt_inf  where cod_mun = 210055 ALLOW FILTERING')
rows = session.execute(query)
rows[:]

[Row(ano=2014, system_count_mes=12, qtd_obt=36, system_sum_vlr_bnf=22396579.0, cod_mun=210005),
 Row(ano=2014, system_count_mes=12, qtd_obt=3, system_sum_vlr_bnf=2913500.0, cod_mun=210010),
 Row(ano=2014, system_count_mes=12, qtd_obt=1, system_sum_vlr_bnf=4784574.0, cod_mun=210015),
 Row(ano=2014, system_count_mes=12, qtd_obt=1, system_sum_vlr_bnf=9942430.0, cod_mun=210020),
 Row(ano=2014, system_count_mes=12, qtd_obt=15, system_sum_vlr_bnf=8545408.0, cod_mun=210030),
 Row(ano=2014, system_count_mes=12, qtd_obt=1, system_sum_vlr_bnf=2895842.0, cod_mun=210040),
 Row(ano=2014, system_count_mes=12, qtd_obt=13, system_sum_vlr_bnf=10858674.0, cod_mun=210043),
 Row(ano=2014, system_count_mes=12, qtd_obt=5, system_sum_vlr_bnf=11160674.0, cod_mun=210047),
 Row(ano=2014, system_count_mes=12, qtd_obt=1, system_sum_vlr_bnf=3111700.0, cod_mun=210050),
 Row(ano=2014, system_count_mes=12, qtd_obt=2, system_sum_vlr_bnf=2744017.0, cod_mun=210055),
 Row(ano=2014, system_count_mes=12, qtd_obt=20, system

In [153]:
# verificando a soma dos valores bnf 
query = """select ano, mes, qtd_obt,vlr_bnf, cod_mun FROM tfbda.bsafm_obt
where cod_mun = 210005 and ano = 2014 GROUP BY ano, cod_mun, mes limit 24 ALLOW FILTERING"""
rows = session.execute(query)
rows[:]
a = 0

for item in rows:
    a += item.vlr_bnf
#     print(item.vlr_bnf)
    
print(a)

22396579.0


In [91]:
# criando o arquivo merged file, que é a juncao do bolsa familia e obitos
import pandas as p

df1 = pd.read_csv('obitos_infantis_MA.csv', sep = ';')
# df = pd.read_csv('novos_dados.csv',encoding ='latin1', error_bad_lines=False, sep='\t', header=0)
df2 = pd.read_csv('bolsa_familia_Brasil.csv', encoding ='latin1', error_bad_lines=False)
# df2.to_csv('bolsa_familia_Brasil.csv', encoding='utf-8')
# df2 = pd.read_csv('bolsa_familia_Brasil.csv')
print(df1.columns)
print(df2.columns)
df3 = pd.merge(df1, df2, on=['cod_mun', 'ano'])
df3 = df3.drop(df3.columns[df3.columns.str.contains('unnamed',case = False)],axis = 1)
df3.to_csv('merged_data.csv')
#select ano, mes, qtd_obt,sum(qtd_fam), cod_mun FROM tfbda.bsafm_obt GROUP BY ano, mes, cod_mun limit 24;

Index(['ano', 'cod_mun', 'num_obt'], dtype='object')
Index(['ano', 'mês', 'cod_mun', 'ano_mes', 'qtd_fam', 'vlr_bnf'], dtype='object')


In [74]:
#cria dataframe
import pandas as pd
import numpy as np

ano = []
municipio = []
valor = []
obitos = []
for item in rows:
    ano.append(item.ano)
    municipio.append(item.cod_mun)
    valor.append(item.system_sum_vlr_bnf)
    obitos.append(item.qtd_obt)

df = pd.DataFrame()

df[0] = ano
df[1] = obitos
df[2] = valor
df[3] = municipio
df.columns = ['ano', 'obitos', 'valor', 'mun']

In [107]:
df[df['mun']==210180]

,ano,obitos,valor,mun
27,2014,1,2663671.0,210180
227,2009,1,954604.0,210180
844,2006,1,538211.0,210180
1253,2007,1,679939.0,210180
1462,2015,1,2791130.0,210180
1861,2004,1,164660.0,210180
2063,2016,1,2944235.0,210180


In [125]:
print(len(pd.unique(df['mun'])))
uniqueMun = pd.unique(df['mun'])

l = []
#calcula coeficiente de correlacao
for item in uniqueMun:
    data = df[df['mun'] == item]
    data = data.sort_values(by = 'ano')
    a  = np.corrcoef((data['valor']), data['obitos'] )
    #cria uma lista de municipio e coeficiente
    l.append(list([item, a[1,0]]))
    
#cria um dataframe(tabela) de municipio e  coeficiente de correlacao
dfcoef = pd.DataFrame(l,columns=list(['mun', 'coef'])) 
dfcoef.to_csv("municipio e correlacao.csv", index = False)

217
Index(['mun', 'coef'], dtype='object')


/home/pedro/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:2400: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/pedro/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:2401: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
